## girvan_newman

In [ ]:
G = nx.Graph(directed = True)
G.add_edges_from(data_abnormal[['source', 'target']].values.tolist())

girvan_newman = nx.algorithms.community.girvan_newman(G=G)

coverage = []
performance = []
modularity = []

for i, partition in enumerate(girvan_newman):
    # n개의 node가 있을 때, n개의 community까지 분화할 수 있으므로,
    # n개로 분화되면 loop 탈출.
    partition_coverage = nx.algorithms.community.coverage(G, partition)
    partition_performance = nx.algorithms.community.performance(G, partition)
    partition_modularity = nx.algorithms.community.modularity(G, partition)
    coverage.append(partition_coverage)
    performance.append(partition_performance) 
    modularity.append(partition_modularity)
    print(
        f"{i:2d} time :: Coverage: {partition_coverage:.4f}, Performance: {partition_performance:.4f}, Modularity: {partition_modularity:.4f}"
    )
    if False:  #print all communities?
        for j, comm in enumerate(partition):
            print(f"community {j:2d} at time {i:2d} ==> {comm}")
print("==" * 20)

## Clauset_Newman_Moore

In [ ]:
G = nx.Graph(directed = True)
G.add_edges_from(data_abnormal[['source', 'target']].values.tolist())

import networkx.algorithms.community as nxcom

Clauset_Newman_Moore= nx.algorithms.community.greedy_modularity_communities(G=G)
communities = sorted(nxcom.greedy_modularity_communities(G), key=len, reverse=True)


print(
   f"Performance: {nx.algorithms.community.performance(G, Clauset_Newman_Moore):.5f}",
    f"Modularity: {nx.algorithms.community.modularity(G, Clauset_Newman_Moore):.5f}",
    f"Coverage: {nx.algorithms.community.coverage(G, Clauset_Newman_Moore):.5f}"
)
for i, comm in enumerate(Clauset_Newman_Moore):
    print(f"community {i:2d} ::: {comm}")
    continue
print("== complete")
print("==" * 30)


def set_node_community(G, communities):
    '''Add community to node attributes'''
    for c, v_c in enumerate(communities):
        for v in v_c:
            # Add 1 to save 0 for external edges
            G.nodes[v]['community'] = c + 1
def set_edge_community(G):
    '''Find internal edges and add their community to their attributes'''
    for v, w, in G.edges:
        if G.nodes[v]['community'] == G.nodes[w]['community']:
            # Internal edge, mark with community
            G.edges[v, w]['community'] = G.nodes[v]['community']
        else:
            # External edge, mark as 0
            G.edges[v, w]['community'] = 0
            
set_node_community(G, communities)
set_edge_community(G)

## Naive_greedy_modularity

In [ ]:
G = nx.Graph(directed = True)
G.add_edges_from(data_abnormal[['source', 'target']].values.tolist())

naive_greedy_modularity = nx.algorithms.community._naive_greedy_modularity_communities(G)

coverage = []
performance = []
modularity = []


for i, partition in enumerate(naive_greedy_modularity):
    # n개의 node가 있을 때, n개의 community까지 분화할 수 있으므로,
    # n개로 분화되면 loop 탈출.
    partition_coverage = nx.algorithms.community.coverage(G, partition)
    partition_performance = nx.algorithms.community.performance(G, partition)
    partition_modularity = nx.algorithms.community.modularity(G, partition)
    coverage.append(partition_coverage)
    performance.append(partition_performance) 
    modularity.append(partition_modularity)
    print(
        f"{i:2d} time :: Coverage: {partition_coverage:.4f}, Performance: {partition_performance:.4f}, Modularity: {partition_modularity:.4f}"
    )
    if False:  #print all communities?
        for j, comm in enumerate(partition):
            print(f"community {j:2d} at time {i:2d} ==> {comm}")
print("==" * 20)

## Louvain

In [ ]:
import community as community_louvain
import community

G = nx.Graph()
G.add_edges_from(data_abnormal[['source', 'target']].values.tolist())
partition = community_louvain.best_partition(G)
modularity = community_louvain.modularity(partition, G)
modularity

In [ ]:
import pandas as pd
node = pd.DataFrame(list(partition.items()), columns=['actor_account_id', 'community'])

---
community featuers
---

In [ ]:
community_index = list(node.community.unique())
for i in community_index:
    community_node = list(node[node.community == i].actor_account_id)
    community_graph = G.subgraph(community_node)
    print(nx.algorithms.assortativity.degree_pearson_correlation_coefficient(community_graph))  ## node가 2개이면 None
    print(nx.algorithms.assortativity.degree_assortativity_coefficient(community_graph))
    print(nx.algorithms.distance_measures.radius(community_graph))
    print(nx.algorithms.distance_measures.radius(community_graph))

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import copy
from networkx.algorithms import community
import matplotlib.animation as animation


def Add_Inner_edges(range, num):
    inner_edges = []
    while len(inner_edges) < num:
        tmp = np.sort(np.random.choice(range, size=2, replace=None)).tolist()
        tmp += [np.random.uniform(0, 1)]  # random weight
        new_edge = tuple(tmp)
        if new_edge not in inner_edges:
            inner_edges = inner_edges + [new_edge]
    return inner_edges


def Add_Outer_edges(Community_all, num):
    # 두 커뮤니티 선택
    outter_edges = []
    while len(outter_edges) < num:
        #group_choiced = np.random.choice(range(len(Community_all)), size=2, replace=None) # 범용적으로 커뮤니티 선택할 시
        if len(outter_edges) < 3:
            group_choiced = np.random.choice([0, 1], size=2, replace=None)
        elif len(outter_edges) < 6:
            group_choiced = np.random.choice([0, 2], size=2, replace=None)
        elif len(outter_edges) < 10:
            group_choiced = np.random.choice([1, 2], size=2, replace=None)
        tmp = np.sort([np.random.choice(Community_all[group_choiced[0]], replace=None),
                 np.random.choice(Community_all[group_choiced[1]], replace=None)]).tolist()
        tmp += [np.random.uniform(0, 1)]  # random weight
        new_edge = tuple(tmp)
        if new_edge not in outter_edges:
            outter_edges = outter_edges + [new_edge]
    return outter_edges


""" Network 생성 """
# 총 150명
G = nx.Graph()
G.add_nodes_from(range(150))
# Community 설정 3개의 커뮤니티
Community1 = range(0, 40)
Community2 = range(40, 90)
Community3 = range(90, 150)
Community_all = Community1, Community2, Community3

# 내부 연결 추가
Inner_edges_1 = Add_Inner_edges(range=Community1, num=100)  # Community 1 100개
Inner_edges_2 = Add_Inner_edges(range=Community2, num=150)  # Community 1 150개
Inner_edges_3 = Add_Inner_edges(range=Community3, num=200)  # Community 1 200개
G.add_weighted_edges_from(Inner_edges_1)
G.add_weighted_edges_from(Inner_edges_2)
G.add_weighted_edges_from(Inner_edges_3)

# 외부 연결 추가
Outter_edges = Add_Outer_edges(Community_all, 50)  # Community 1-2-3 간에 50개
G.add_weighted_edges_from(Outter_edges)
pos = nx.spring_layout(G)

In [ ]:
import community as lvcm


""" Louvain method """
partition = lvcm.best_partition(graph=G, partition=None, weight='weight', resolution=1., randomize=True)
max_k_w = []
for com in set(partition.values()):
    list_nodes = [nodes for nodes in partition.keys()
                  if partition[nodes] == com]
    max_k_w = max_k_w + [list_nodes]

""" Make Community Color list """
community_num_group = len(max_k_w)
color_list_community = [[] for i in range(len(G.nodes()))]
for i in range(len(G.nodes())):
    for j in range(community_num_group):
        if i in max_k_w[j]:
            color_list_community[i] = j

""" Plot Community """
fig = plt.figure()
edges = G.edges()
weights = [G[u][v]['weight'] for u, v in edges]
Feature_color_sub = color_list_community
node_size = 50
im = nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color=Feature_color_sub, cmap='jet', vmin=0, vmax=community_num_group)
nx.draw_networkx_edges(G, pos, width=weights)
nx.draw_networkx_labels(G, pos, font_size=5, font_color="black")
plt.xticks([])
plt.yticks([])
plt.colorbar(im)
plt.show(block=False)

In [ ]:
%matplotlib inline

def ANI_Undirected_plot(time, G, q, color_list, pos, weight, fig):
    global step

    all_step = len(color_list)
    step_view = step % all_step

    subplot = fig.add_subplot(1, 2, 1)
    plt.cla()

    Feature_color_sub = color_list[step_view]
    Gedges = G.edges()
    node_size = 50
    if weight==True:
        weights = [G[u][v]['weight'] for u, v in Gedges]
        nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color=Feature_color_sub, cmap='jet', vmin=0, vmax=len(set(Feature_color_sub)))
        nx.draw_networkx_edges(G, pos, width=weights)
    else:
        nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color=Feature_color_sub, cmap='jet', vmin=0, vmax=len(set(Feature_color_sub)))
        nx.draw_networkx_edges(G, pos)
    nx.draw_networkx_labels(G, pos, font_size=5, font_color="black")
    plt.xticks([])
    plt.yticks([])
    plt.title('Community')

    subplot = fig.add_subplot(1, 2, 2)
    plt.scatter(step_view, q[step_view], c='blue')
    plt.title('Modularity')

    plt.suptitle('step: ' + str(step_view))

    if step < all_step-2:
        step = step + 1


""" Make Community Color list """
color_list_community_all = []
for communities_result in log_community_all:
    community_num_group = len(communities_result)
    color_list_community = [[] for i in range(len(G.nodes()))]
    for i in range(len(G.nodes())):
        for j in range(community_num_group):
            if i in communities_result[j]:
                color_list_community[i] = j
    color_list_community_all += [color_list_community]

""" Modularity """
t = []
q = []
for i in range(len(log_community_all)):
    Q = community.modularity(G, communities=log_community_all[i], weight='weight')
    t += [i]
    q += [Q]

""" Plot Community (Dynamic) """
fig_ani = plt.figure(figsize=(14, 6))
step = 0
weight = False
ani = animation.FuncAnimation(fig_ani, ANI_Undirected_plot, fargs=(G, q, color_list_community_all, pos, weight, fig_ani),
                              interval=5, save_count=len(q))

from IPython.display import HTML
                   
HTML(ani.to_jshtml())   

ani.to_html5_video()